Name: Rachael Muriuki<br>
StudentID: CS-DA01-25122<br>
Assignment: Natural Language Processing with Transformers

1.Importing Required Libraries

In [ ]:
!pip install transformers #Installing hugging face transformers

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

2.Load Pretrained BERT

In [ ]:
#Loading pre-trained BERT tokenizer and model from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were 

3.Sentence Pairs & Ground Truth labels

In [ ]:
sentence_pairs = [
    ("How do i learn Python?", "What is the best way to study Python?"),
    ("What is AI?", "How to cook pasta?"),
    ("How do i bake a chocolate cake?", "Give me a chocoalte cake recipe."),
    ("How can i improve my coding skills?", "Tips for becoming better at programming."),
    ("Where can i buy cheap laptops?", "Best sites to find affordable computers."),
    ("I'm going to the bank to deposit money.", "She sat by the river bank."),
    ("I enjoy watching football.", "Soccer is my favorite sport."),
    ("She is playing the bass guitar.", "The bass swam in the lake."),
    ("What time does the movie start?", "When will the film begin?"),
    ("Do you like coffee?", "I prefer tea over soda."),
    ]

#Ground truth similarity labels: 1 = similar, 0 = not similar
labels = [1, 0, 1, 1, 1, 0, 1, 0, 1, 0]

4.Embedding function

In [ ]:
#Function to get the BERT [CLS] embediing for a sentence
def get_sentence_embedding(sentence):

  #Tokenize and encode sentence into input tensors
  inputs = tokenizer(sentence, return_tensors = 'tf', padding = True, truncation = True)

  #Get model output
  outputs = bert_model(inputs)

  #Extract [CLS] token embedding (shape: [1, 768])
  cls_embedding = outputs.last_hidden_state[:, 0, :]
  return cls_embedding.numpy()

5.Prediction loop

In [ ]:
#Calculate cosine similarity for each pair
predictions = []

for sent1, sent2 in sentence_pairs:
  emb1 = get_sentence_embedding(sent1)
  emb2 = get_sentence_embedding(sent2)
  sim_score = cosine_similarity(emb1, emb2)[0][0]
  pred = 1 if sim_score > 0.7 else 0
  predictions.append(pred)

  print(f"\nSentence 1: {sent1}")
  print(f"Sentence 2: {sent2}")
  print(f"Cosine Similarity: {sim_score:.4f} → Predicted Similar: {pred}")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.



Sentence 1: How do i learn Python?
Sentence 2: What is the best way to study Python?
Cosine Similarity: 0.9743 → Predicted Similar: 1

Sentence 1: What is AI?
Sentence 2: How to cook pasta?
Cosine Similarity: 0.9033 → Predicted Similar: 1

Sentence 1: How do i bake a chocolate cake?
Sentence 2: Give me a chocoalte cake recipe.
Cosine Similarity: 0.9072 → Predicted Similar: 1

Sentence 1: How can i improve my coding skills?
Sentence 2: Tips for becoming better at programming.
Cosine Similarity: 0.8633 → Predicted Similar: 1

Sentence 1: Where can i buy cheap laptops?
Sentence 2: Best sites to find affordable computers.
Cosine Similarity: 0.8750 → Predicted Similar: 1

Sentence 1: I'm going to the bank to deposit money.
Sentence 2: She sat by the river bank.
Cosine Similarity: 0.8198 → Predicted Similar: 1

Sentence 1: I enjoy watching football.
Sentence 2: Soccer is my favorite sport.
Cosine Similarity: 0.9348 → Predicted Similar: 1

Sentence 1: She is playing the bass guitar.
Sentence

6.Accuracy Calculation

In [ ]:
correct = sum([pred == label for pred, label in zip(predictions, labels)])
accuracy = correct / len(labels)
print(f"\nFinal Accuracy: {accuracy:.2%}")


Final Accuracy: 60.00%
